# Instructions

What you're looking at is a Jupyter notebook. (If you knew that already, you don't need to read the rest of this cell. Indeed, you won't need to read most of the markdown cells.)

This notebook is a tool for running code from your browser. The code exists in various cells, wich you should run one-at-a-time by hitting shift+enter. 

The cell below this one gives you an opportunity to define your own parameters, or leave them as defaults. Click on the cell and hit shift+enter. Below the cell, there should be some buttons and entry fields with default values that you can change. 

If you're comfortable doing so, you may also change the default values in the code below. This has the advantage of being saveable.

In [1]:
# Imports
import math
import importlib
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, Button, Box, VBox, Label

adefaultvalue=0.8333 ### stitch height
ddefaultvalue=10     ### number of stitches in a straight segment
wdefaultvalue=0.5    ### width of a strand, as a proportion of the length of a straight segment
QTSdefaultvalue=0.3  ### number of straight stitches to add to each end of a quarter turn
Tsdefaultvalue=0      ### number of straight stitches to stretch a twist by

a=widgets.FloatText(value=adefaultvalue, disabled=False)
print("Enter your single crochet height:")
display(a)

d=widgets.IntText(value=ddefaultvalue, disabled=False)
print("Enter your dot-to-dot length:")
display(d)

w=widgets.FloatSlider(value=wdefaultvalue,disabled=False,min=0,max=1)
print("How wide should a strand be, as a proportion of its dot-to-dot length?")
display(w)

QTS=widgets.FloatSlider(value=QTSdefaultvalue,disabled=False,min=0,max=1)
print("How long should the straight section on a quarter turn be, as a proportion of the dot-to-dot length?")
display(QTS)

Ts=widgets.IntText(value=Tsdefaultvalue, disabled=False)
print("How many straight stitches do you want to add to a twist, to make it taller?")
display(Ts)

#####################

button = widgets.Button(description="restore defaults")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    a.value=adefaultvalue
    d.value=ddefaultvalue
    w.value=wdefaultvalue
    QTS.value=QTSdefaultvalue
    Ts.value=Tsdefaultvalue

button.on_click(on_button_clicked)



How long should the straight section on a quarter turn be, as a proportion of the dot-to-dot length?


FloatSlider(value=0.3, max=1.0)

How many straight stitches do you want to add to a twist, to make it taller?


IntText(value=0)

Button(description='restore defaults', style=ButtonStyle())

Output()

# After you've made any desired changes above, the cell below will tell you what values the program will be using.

Click on it and hit shift+enter, like with all the code cells

In [2]:
sch=a.value
print("single crochet height:",sch)
if a.value>1:
        print("Generally single crochets are shorter than they are wide.")
elif a.value<.5:
        print("That is a very short stitch.")

dtd=d.value
print()
print("dot-to-dot length:",dtd,"stitches")
if d.value<3:
    print("that length is too short")
    
QTStraight=QTS.value
widthinstitches=dtd*w.value
widthinrows=widthinstitches/sch
HalfRows=round(widthinrows/2)#needs to be an even number of rows
TotalRows=2*HalfRows
QTs=round(QTStraight*dtd)
TS=Ts.value

print()
print("number of rows:",TotalRows)

print()
print("Straight bits:", QTs,"in a quarter turn,",TS,"in a twist")

single crochet height: 0.8333

dot-to-dot length: 10 stitches

number of rows: 6

Straight bits: 3 in a quarter turn, 0 in a twist


# Check Gauge and Proportions

Now that your basic values are set, the cell below will check to make sure they are reasonable relative to one another.

In [3]:
warnings=0

##check proportions
###############################################
#max distance from the centre of the strand is Delta
#basically widthinstitches/2, but accounting for rounding:
Delta = HalfRows*sch

#quarter turn's straight segment's total length should not exceed u-(sqrt 2 - 1)Delta,
#where Delta is the max distance from the middle of the strand to the edge
maxs = dtd-2.414*Delta
if QTs > maxs:
    print()
    print("*Warning*")
    print(" A quarter turn might overrun its boundaries. Max length of the quarter straight section is",round(maxs,2))
    warnings=warnings+1
    
#Delta shouldn't exceed dtd*sch*(4-2\sqrt 2), or else a point might overrun
maxp=dtd*sch*1.17
if Delta>maxp:
    print()
    print("*Warning*")
    print("A point might overrun its boundaries. To avoid, decrease width of strand.")
    warnings=warnings+1
    

#Delta shoudn't exceed dtd(sqrt 2 - 1), or else the  twists overrun their feature walls
maxd = dtd*(2**.5-1)
if Delta > maxd:
    print()
    print("*Warning*")
    print("A twist might overrun its boundaries. To avoid, decrease width of strand.")
    warnings=warnings+1
    
#twist extra straight part shouldn't exceed (sqrt 2 -1)*dtd-Delta
maxt=(2**.5-1)*dtd-Delta
if TS > maxt:
    print()
    print("*Warning*")
    print("A twist might be too tall. To avoid, decrease the straight stitches added to twists.")
    warnings=warnings+1
    

#Delta shouldn't exceed dtd/sqrt 8, or else the J-type and C-type corners will overrun their feature walls
#Runs aren't a standard component, so for many knots, this warning doesn't apply
maxj = dtd/8**.5
if Delta > maxj:
    print()
    print("*Possible Warning*")
    print("If you plan to use a run with a J-type or C-type corner, it might overrun its boundaries. To avoid, decrease how wide a strand should be.")
    print("If your knot does not have these components, disregard")
    warnings=warnings+1
if warnings==0:
    print("No warnings.")

No warnings.


# Generate Strings

Now, the program will figure out the basic lengths of each row for each type of standard component, using the values you've set above. 

This will print out some lists of lengths, which are sometimes nice to refer to for trouble-shooting.

In [4]:
####straight
SCount=dtd*(TotalRows+1)
SOCount=2*(dtd)
print("Straight:",SCount,"stitches and ",SOCount,"outline stitches")
print()

#####quarter turn
QuarterTurn=[]
QuarterTurnStitchCount=0


for r in range(-HalfRows,HalfRows+1):
    RadiusShift=sch*r
    Radius=dtd+RadiusShift-QTs
    UnroundedArcLength = math.pi/2*Radius
    ArcLength = round(UnroundedArcLength)
    QuarterTurn.append(ArcLength)
    QuarterTurnStitchCount=QuarterTurnStitchCount+ArcLength
    if ArcLength<0:
        print("Something has gone wrong: quarter turns have a negative arclength")
QCount=sum(QuarterTurn)
QOCount=QuarterTurn[0]+QuarterTurn[-1]
print("Quarter turn:",QuarterTurn,";",QCount,"stitches total;",QOCount,"outline stitches")
print()

#####point
PointArc=[]
PointCorner=[]

S=round(dtd*(1-(.5**.5)))
S=max(S,HalfRows+1)

for r in range(-HalfRows,HalfRows+1):
    #arc
    RadiusShift=sch*r
    Radius=dtd+RadiusShift
    UnroundedArcLength = math.pi/4*Radius
    ArcLength = round(UnroundedArcLength)
    PointArc.append(ArcLength)
    if ArcLength<0:
        print("Something has gone wrong: points have a negative arclength")
        
    #corner
    Side=S+r
    PointCorner.append(Side)
    if Side<0:
        print("Something has gone wrong: points are too short on their inside corners")
PCount=2*sum(PointArc)+2*sum(PointCorner)
POCount=2*(PointArc[0]+PointArc[-1]+PointCorner[0]+PointCorner[-1])+1
print("PointArc:",PointArc)
print("PointCorner:",PointCorner)
print(PCount,"stitches and",POCount,"outline stitches")
print()

#####twist
###sides are the same arcs as points, so we don't generate them again
TwistTop=[]
for r in range(-HalfRows,HalfRows+1):
    #arcradius
    RadiusShift=sch*r
    Radius=dtd+RadiusShift
    #halfcircle radius
    HRadius = dtd*(1-math.sqrt(.5))+RadiusShift
    UnroundedArcLength = math.pi*HRadius
    ArcLength = round(UnroundedArcLength)
    TwistTop.append(max(ArcLength,1))

for sts in range (1,len(TwistTop)):
    difference=TwistTop[sts]/TwistTop[sts-1]
    if difference>2:
        TwistTop[sts-1]=math.ceil(TwistTop[sts]/2)

TCount=sum(TwistTop)+2*sum(PointArc)
TOCount=2*(PointArc[0]+PointArc[-1])+TwistTop[0]+TwistTop[-1]
print("TwistTop:",TwistTop,";",TCount,"stitches and",TOCount,"outline stitches")
print()

################
#now, for runs
################
dtype=[]
#lenghts of arc. 
for r in range(-HalfRows,HalfRows+1):
    turn=round(math.pi/4*(dtd/0.8284+r*sch))
    dtype.append(turn)
    if turn<0:
        print("Something has gone wrong: D-type corners have a negative arclength")
DCount=sum(dtype)
DOCount=dtype[0]+dtype[-1]
print("D:",dtype,";",DCount,"stitches; ",DOCount,"outline stitches")
#dtype also has straight bit 0.9142*dtd, same every row
daddstraight=0.2071*dtd

#jtype: lengths of arc. 1: quarter turn (next to straight section). 2: eighth turn (starts or ends run).
jtype1=[]
jtype2=[]


#jaddstraight: straight lengths adjacent to run:
#each row the same; don't round because it will be added to middle straight part
jaddstraight=dtd/2

#The j-type lists are made in two stages.
#For chunky dimensions, it's possible for one arclength to be negative and the other positive.
#In this case, add them together to have a total arclength as close as possible to the ideal length
for r in range(-HalfRows,HalfRows+1):
    turn1temp=math.pi/2*(dtd*0.2071+r*sch)
    turn2temp=math.pi/4*(dtd*0.8536+r*sch)
    if turn1temp<0:
            turn1=0
            turn2=max(round(turn1temp+turn2temp),0)
    elif turn2temp<0:
            turn2=0
            turn1=max(round(turn1temp+turn2temp),0)
    else:
        turn1=round(turn1temp)
        turn2=round(turn2temp)
    jtype1.append(turn1)
    jtype2.append(turn2)
    if turn1<0 or turn2<0:
        print("Something has gone wrong: J-type corners have a negative arclength")
JCount=sum(jtype1)+sum(jtype2)+(TotalRows+1)*round(jaddstraight)
JOCount=jtype1[0]+jtype1[-1]+jtype2[0]+jtype2[-1]+round(2*jaddstraight)
print("J straight:",jaddstraight)
print("J:",jtype1,jtype2,";",JCount,"stitches and ",JOCount,"outline stitches")


#L-type
#straight: subtract 1, becuase ch / dec will be specified explicitly
#corner instructions the same for every row (2ch or 2 dec)
#straight instructions different for every row
#this is the only corner type with changing straights
laddstraight=[]
for r in range(-HalfRows,HalfRows+1):
    laddstraight.append(round(dtd/1.414+r*sch-1,2))
    if round(dtd/1.414+r*sch-1,2)<0:
        print("Something has gone wrong: L-type corners have negative straight length")
LCount=round(2*sum(laddstraight))
LOCount=round(2*(laddstraight[0]+laddstraight[-1]))
print("L straight:",laddstraight,";",LCount,"stitches and ",LOCount,"outline stitches")

#C-type
caddstraight=round(dtd/8**.5,2)
ctype=[]
for r in range(-HalfRows,HalfRows+1):
    cturn=max(0,round(math.pi*(caddstraight+r*sch)))
    ctype.append(cturn)
CCount=round(sum(ctype)+2*(TotalRows+1)*caddstraight)
COCount=round(ctype[0]+ctype[-1]+4*caddstraight)
print("C straight:",caddstraight)
print("C:",ctype,";",CCount,"stitches and",COCount,"outline stitches")

#Total over the entire pattern. Will be added in two places: the run function, and after the pattern instructions are printed.
#For each run, we'll have to add the straight bit. There is some imprecition involved because we are rounding differently for stitch counts than for the instructions, but it should be minor.
TotalCount = 0
TotalOCount = 0

Straight: 70 stitches and  20 outline stitches

Quarter turn: [7, 8, 10, 11, 12, 14, 15] ; 77 stitches total; 22 outline stitches

PointArc: [6, 7, 7, 8, 9, 9, 10]
PointCorner: [1, 2, 3, 4, 5, 6, 7]
168 stitches and 49 outline stitches

TwistTop: [2, 4, 7, 9, 12, 14, 17] ; 177 stitches and 51 outline stitches

D: [8, 8, 9, 9, 10, 11, 11] ; 66 stitches;  19 outline stitches
J straight: 5.0
J: [0, 1, 2, 3, 5, 6, 7] [4, 5, 6, 7, 7, 8, 9] ; 105 stitches and  30 outline stitches
L straight: [3.57, 4.41, 5.24, 6.07, 6.91, 7.74, 8.57] ; 85 stitches and  24 outline stitches
C straight: 3.54
C: [3, 6, 9, 11, 14, 16, 19] ; 128 stitches and 36 outline stitches


# Runs

The cell below defines a program for adding runs to your pattern. 

## non-technical instructions
Hit shift+enter on the cell below

## slightly technical instructions

A *single component* can consist of one or more runs. 

* There are four corner types, each of which can be inner or outer: D, J, L, C
  * D and J only occur at the start or end of a component.
  * L and C only occur between two runs, inside one component
* Between each corner type, count the number of feature walls that make up the run. That is, the combined number of feature walls on the two adjacent parallel lines of feature walls
* The function "run" defined below will return a string of instructions, one entry per row, for a single component. 
  * When generating the pattern for an entire strand, you can pass a string of instructions, one entry per row
  * You can have the function called as part of your pattern, e.g. pattern=["S",run("OD",5,"OD"),"S"]
  * Or, you can first generate strings, e.g. R=run("OD",5,"OD") pattern=["S",R,"S"]

The syntax for describing a run to plug into the run function is: alternating strings and ints. 
*  Strings: OD, ID, OJ, IJ, OL, IL, OC, IC
*  Ints: must be at least 2; otherwise it's not a run
*  Program will print an error message if there's a physical problem:
   * When an end / middle component is used in the middle / end, and
   * when the outer/inner designations don't match with the lengths



In [5]:
###
#input: a name, and a description of the run
#descriptions: list of alternating strings and ints
#strings: ID, OD, IJ, OJ, IL, OL, IC, OC
#function will return a string; entries are instructions for each row
#this way, can put run instructions inside pattern=[] string


def run(run_input):
    global TotalCount
    global TotalOCount

    instructions=[]
    ##separate out corners and lengths
    corners=[]
    lengths=[]
    for i in range(len(run_input)):
        if i%2==0:
            corners.append(run_input[i])
            #also add in corner counts to total stitch counts
            if run_input[i]=="ID" or run_input[i]=="OD":
                TotalCount=TotalCount+DCount
                TotalOCount=TotalOCount+DOCount
            if run_input[i]=="IJ" or run_input[i]=="OJ":
                TotalCount=TotalCount+JCount
                TotalOCount=TotalOCount+JOCount
            if run_input[i]=="IC" or run_input[i]=="OC":
                TotalCount=TotalCount+CCount
                TotalOCount=TotalOCount+COCount
        else:
            lengths.append(run_input[i])
            #add straight parts, which were not accounted for when we generated the strings
            TotalCount=TotalCount+round((TotalRows+1)*dtd*(run_input[i]-2)/1.414)
            TotalOCount=TotalOCount+round(2*dtd*(run_input[i]-2)/1.414)

    #now the corners and lengths are defined
    #check that corners start and end appropriately
    #doing this here rather than later because the later iteration doesn't make it to the last index
    if corners[0][1]!="D" and corners[0][1]!="J":
        print("Segment must start with D or J")
    if corners[len(corners)-1][1]!="D" and corners[len(corners)-1][1]!="J":
        print("Segment must end with D or J")
    for m in range(1,len(corners)-1):
        if corners[m][1]=="D" or corners[m][1]=="J":
            print("Runs are joined by C or L type corners, not D or J. Check join",m)

    for r in range(-HalfRows,HalfRows+1):
        row=r+HalfRows
        if row==0:
            stitch="ch"
        else:
            stitch="sc"
        delta=sch*r
        ###stitch height * row, where delta=0 in the centre of the strand
        instructions.append("")#add a blank string to start off the instructions for this row
        skipnextcorner=0 #indicator: did we just do a C? If so, the next corner has no instructions, so don't make a blank / / for it
##########################################################################################
        for m in range(len(lengths)): ##iterate over each run in the segment
##########################################################################################
            #the current run has corner1 end type, length feature walls in the run, and corner2 end type
            corner1=corners[m]
            orientation1=corner1[0]
            corner2=corners[m+1]
            orientation2=corner2[0]
            length=lengths[m]
    ############################################################################
            #Parity check: even-lenths change orientation, odd-lengths preserve orientation
            if length%2==0:
                if orientation1==orientation2:
                    print("Error: an even-length run has opposite orientations on each end  (one inner, one outer)")
            else:
                if orientation1!=orientation2:
                    print("Error: an odd-length run has the same orientations on each end  (both inner, or both outer)")
    #############################################################################
    #middle
            straightmiddle=(length-2)/1.414*dtd
    #first corner
    ############################################################################
            #straight1:  straight segment associated with first corner. Will be added to straightmiddlepart
            if corner1=="ID" or corner1=="OD":
                straight1=daddstraight
                if corner1=="ID":##inside to outside
                    turnlength=dtype[row]
                    if r!=-HalfRows:
                        difference=dtype[row]-dtype[row-1]
                        if difference==0:
                            change=""
                        else:
                            change="(+"+str(difference)+")"
                    else:
                        change=""

                else:##outside to inside
                    turnlength=dtype[TotalRows-row]
                    if r!=-HalfRows:
                        difference=dtype[TotalRows-row+1]-dtype[TotalRows-row]
                        if difference==0:
                            change=""
                        else:
                            change="(-"+str(difference)+")"
                    else:
                        change=""

                corner1instructions=str(turnlength)+" "+stitch+change

            elif corner1=="IJ" or corner1=="OJ":
                straight1=jaddstraight#since this is the *first* corner, it starts outside the run
                if corner1=="IJ": #inside to outside
                    if row!=0:
                        difference2=jtype2[row]-jtype2[row-1]
                        difference1=jtype1[row]-jtype1[row-1]
                        if difference2!=0:
                            change2=" (+"+str(difference2)+")"
                            if jtype2[row-1]==0:
                                change2=" (+"+str(difference2)+", new segment -- might want to work chains instead of sc)"
                        else:
                            change2=""
                        if difference1!=0:
                            change1=" (+"+str(difference1)+")"
                            if jtype1[row-1]==0:
                                change1=" (+"+str(difference1)+", new segment -- might want to work chains instead of sc)"
                        else:
                            change1=""
                    else:
                        change2=""
                        change1=""
                    if row%2==0:
                        corner1instructions=str(jtype2[row])+" "+stitch+change2+" / "+str(jtype1[row])+" "+stitch+change1
                    else:
                        corner1instructions=str(jtype1[row])+" "+stitch+change1+" / "+str(jtype2[row])+" "+stitch+change2
                else:#outside to inside
                    if row!=0:
                        difference2=jtype2[TotalRows-row+1]-jtype2[TotalRows-row]
                        difference1=jtype1[TotalRows-row+1]-jtype1[TotalRows-row]
                        if difference2!=0:
                            change2="(-"+str(difference2)+")"
                            if jtype2[TotalRows-row]==0:
                                change2="(-"+str(difference2)+", end segment)"
                        else:
                            change2=""
                        if difference1!=0:
                            change1="(-"+str(difference1)+")"
                            if jtype1[TotalRows-row]==0:
                                    change1="(-"+str(difference1)+", end segment)"
                        else:
                            change1=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner1instructions=str(jtype2[TotalRows-row])+" "+stitch+change2+" / "+str(jtype1[TotalRows-row])+" "+stitch+change1
                    else:
                        corner1instructions=str(jtype1[TotalRows-row])+" "+stitch+change1+" / "+str(jtype2[TotalRows-row])+" "+stitch+change2
            elif corner1=="IL" or corner1=="OL":
                if corner1=="IL":
                    straight1=laddstraight[row]
                    corner1instructions="2 ch "
                else:
                    straight1=laddstraight[TotalRows-row]
                    corner1instructions="2 dec "
            elif corner1=="IC" or corner1=="OC":
                straight1=caddstraight
                if corner1=="IC":#inside to outside
                    if row!=0:
                        difference=ctype[row]-ctype[row-1]
                        if difference!=0:
                            change="(+"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner1instructions=str(ctype[row])+" "+stitch+change
                else: 
                    if row!=0:
                        difference=ctype[TotalRows-row+1]-ctype[TotalRows-row]
                        if difference!=0:
                            change="(-"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                corner1instructions=str(ctype[TotalRows-row])+" "+stitch+change
            else:
                print("unknown corner1 type")

    #second corner
    ############################################################################
            #straight2:  straight segment associated with second corner. Will be added to straightmiddlepart        
            if corner2=="ID" or corner2=="OD":
                straight2=daddstraight
                if corner2=="ID":#inside to outside
                    if row!=0:
                        difference=dtype[row]-dtype[row-1]
                        if difference!=0:
                            change="(+"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner2instructions=str(dtype[row])+" "+stitch+" "+change
                else: #outside to inside
                    if row!=0:
                        difference=dtype[TotalRows-row+1]-dtype[TotalRows-row]
                        if difference!=0:
                            change="(-"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner2instructions=str(dtype[TotalRows-row])+" "+stitch+change
            elif corner2=="IJ" or corner2=="OJ":
                straight2=jaddstraight
                if corner2=="IJ":#in to out
                    if row!=0:
                        difference1=jtype1[row]-jtype1[row-1]
                        difference2=jtype2[row]-jtype2[row-1]
                        if difference1!=0:
                            change1=" (+"+str(difference1)+")"
                            if jtype1[row-1]==0:
                                change1=" (+"+str(difference1)+", new segment -- might want to work chains instead of sc)"
                        else:
                            change1=""
                        if difference2!=0:
                            change2=" (+"+str(difference2)+")"
                            if jtype2[row-1]==0:
                                change2=" (+"+str(difference2)+", new segment -- might want to work chains instead of sc)"
                        else:
                            change2=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner2instructions=str(jtype1[row])+" "+stitch+change1+" / "+str(jtype2[row])+" "+stitch+change2
                    else:
                        corner2instructions=str(jtype2[row])+" "+stitch+change2+" / "+str(jtype1[row])+" "+stitch+change1
                else: #out to in
                    if row!=0:
                        difference1=jtype1[TotalRows-row+1]-jtype1[TotalRows-row]
                        difference2=jtype2[TotalRows-row+1]-jtype2[TotalRows-row]
                        if difference1!=0:
                            change1="(-"+str(difference1)+")"
                            if jtype1[TotalRows-row]==0:
                                change1="(-"+str(difference1)+", end segment)"
                        else:
                            change1=""
                        if difference2!=0:
                            change2="(-"+str(difference2)+")"
                            if jtype2[TotalRows-row]==0:
                                change2="(-"+str(difference2)+", end segment)"
                        else:
                            change2=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner2instructions=str(jtype1[TotalRows-row])+" "+stitch+change1+" / "+str(jtype2[TotalRows-row])+" "+stitch+change2
                    else:
                        corner2instructions=str(jtype2[TotalRows-row])+" "+stitch+change2+" / "+str(jtype1[TotalRows-row])+" "+stitch+change1
            elif corner2=="IL" or corner2=="OL":            
                corner2instructions="" #the 2ch or 2 dec is written when it's corner1
                if corner2=="IL":
                    straight2=laddstraight[row]
                else:
                    straight2=laddstraight[TotalRows-row]
            elif corner2=="IC" or corner2=="OC":
                straight2=caddstraight

            else:
                print("unknown corner2 type")
    ##########################################
    #straight middle
    ##########################################
            straighttotal=straight1+straightmiddle+straight2
            straightstitches=round(straighttotal)
    ###########################################
    #segment instructions
    ############################################
            if row%2==0:
                if m==len(lengths)-1:#last segment: don't add a final / 
                    if corner2[1]=="C" or corner2[1]=="L":#no corner2
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch
                    else:
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+"  / "+corner2instructions
                else:
                    if corner2[1]=="C" or corner2[1]=="L":#no corner2
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+" / "
                    else:
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner2instructions+" / "
            else:#add to the LEFT, because we're going right-to-left
                if m==0: #first segment, which is shown last because we're going right-to-left:
                    if corner2[1]=="C" or corner2[1]=="L":
                        instructions[row]=str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]
                    else:
                        instructions[row]=corner2instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]
                else:
                    if corner2[1]=="C" or corner2[1]=="L":
                        instructions[row]=str(straightstitches)+" "+stitch+" / "+corner1instructions+" / "+instructions[row]
                    else:
                        instructions[row]=corner2instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner1instructions+" / "+instructions[row]
    return instructions



# Pattern

Now you need to tell the program what your knot looks like. 

## Non-technical instructions

If you're wary of programming, use shift+enter on the cell below. You'll see a list of buttons pop up. These will allow you to enter the components of your knot. One at a time, click on a component to add it to your pattern.

If you want to add a run, choose the corner types and the length, then click "add run."

At this time we can't accommodate runs with C-type or L-type corners using this entry widget.

## Slightly technical instructions

The components of your knot need to be stored in a string named "pattern." The program knows standard components, and a little bit about runs. Probalby the most efficient way of entering these is to explicitly define the string yourself. Skip to the cell called "Patterns and Markers" to do this.



In [6]:
print("Click on your components in order.")
print("If you have a run with a C-type or L-type corner, you need to enter that into your pattern directly, rather than using these widgets. Sorry! I'm still learning.")

pattern=[]
displaypattern=[]
output = widgets.Output()


Sbutton = widgets.Button(description="straight")
IQbutton = widgets.Button(description="inner quarter turn")
OQbutton = widgets.Button(description="outer quarter turn")
IPbutton = widgets.Button(description="inner point")
OPbutton = widgets.Button(description="outer point")
ITbutton = widgets.Button(description="inner twist")
OTbutton = widgets.Button(description="outer twist")

def on_Sbutton_clicked(b):
    pattern.append("S")
    displaypattern.append("S")
    with output:
        print("straight")
Sbutton.on_click(on_Sbutton_clicked)

def on_IQbutton_clicked(b):
    pattern.append("IQ")
    displaypattern.append("IQ")
    with output:
        print("inner quarter")
IQbutton.on_click(on_IQbutton_clicked)

def on_OQbutton_clicked(b):
    pattern.append("OQ")
    displaypattern.append("OQ")
    with output:
        print("outer quarter")
OQbutton.on_click(on_OQbutton_clicked)

def on_IPbutton_clicked(b):
    pattern.append("IP")
    displaypattern.append("IP")
    with output:
        print("inner point")
IPbutton.on_click(on_IPbutton_clicked)

def on_OPbutton_clicked(b):
    pattern.append("OP")
    displaypattern.append("OP")
    with output:
        print("outer point")
OPbutton.on_click(on_OPbutton_clicked)

def on_ITbutton_clicked(b):
    pattern.append("IT")
    displaypattern.append("IT")
    with output:
        print("inner twist")
ITbutton.on_click(on_ITbutton_clicked)

def on_OTbutton_clicked(b):
    pattern.append("OT")
    displaypattern.append("OT")
    with output:
        print("outer twist")
OTbutton.on_click(on_OTbutton_clicked)

        
###################################################
run1drop=widgets.Dropdown(
    options=["Inner D-type","Inner J-type","Outer D-type", "Outer J-type"],
    value="Inner D-type",
    description='First corner:',
    disabled=False,
)

style = {'description_width': 'initial'}
run3drop=widgets.Dropdown(
    options=["Inner D-type","Inner J-type","Outer D-type", "Outer J-type"],
    value="Inner D-type",
    description='Second corner:',
    style=style,
    disabled=False,
)

run2int=widgets.BoundedIntText(
    value="3",
    min=2,
    description='Length:'
)

runbutton = widgets.Button(description="add run")

def on_runbutton_clicked(b):

    if run1drop.value=="Inner D-type":
        corner1="ID"
    elif run1drop.value=="Inner J-type":
        corner1="IJ"
    elif run1drop.value=="Outer D-type":
        corner1="OD"
    elif run1drop.value=="Outer J-type":
        corner1="OJ"
    if run3drop.value=="Inner D-type":
        corner2="ID"
    elif run3drop.value=="Inner J-type":
        corner2="IJ"
    elif run3drop.value=="Outer D-type":
        corner2="OD"
    elif run3drop.value=="Outer J-type":
        corner2="OJ"


    displaypattern.append("run(["+'"{}"'.format(corner1)+","+str(run2int.value)+","+'"{}"'.format(corner2)+")]")
    pattern.append(run([corner1,run2int.value,corner2]))
    with output:
        print("run: "+corner1+","+str(run2int.value)+","+corner2)
        #print("Pattern:",displaypattern)
runbutton.on_click(on_runbutton_clicked)


###################################################

deletebutton = widgets.Button(description="delete last", button_style='danger')


def on_deletebutton_clicked(b):
    if len(pattern)!=0:
        del pattern[-1]
        del displaypattern[-1]
    with output:
        print("Pattern:",displaypattern)
deletebutton.on_click(on_deletebutton_clicked)



showbutton = widgets.Button(description="show pattern", button_style='info')

def on_showbutton_clicked(b):
    with output:
        print("Pattern:",displaypattern)
showbutton.on_click(on_showbutton_clicked)


###################################################


style = {'description_width': 'initial'}
left_box = widgets.VBox([Sbutton,IQbutton,OQbutton,IPbutton,OPbutton,ITbutton,OTbutton])
right_box = widgets.VBox([run1drop,run2int,run3drop,runbutton,deletebutton,showbutton])
widgets.HBox([left_box,right_box,output])



Click on your components in order.
If you have a run with a C-type or L-type corner, you need to enter that into your pattern directly, rather than using these widgets. Sorry! I'm still learning.


# Markers

To help you assemble the knot, it's nice to number your intersections. Each component of a knot starts and ends at an intersection. If you want to use labelled markers, you can enter them using the widget below. You must already have your pattern entered above.

Alternately, if you're comfortable editing code, skip this cell and define the markers in the next one.

In [13]:
markers=[]

print("Enter the marker labels:")

markernumber=widgets.Text(
    value='1T',
    description='Label:',
    disabled=False
)
###################################################
addmarkerbutton = widgets.Button(description="add label")

output = widgets.Output()

def on_addmarkerbutton_clicked(b):
    if len(markers)<len(pattern)-1:
        markers.append(markernumber.value)
        with output:
            print(markernumber.value)
    else:
        print("The number of markers should be smaller than the number of components in the pattern")
addmarkerbutton.on_click(on_addmarkerbutton_clicked)

###################################################

deletemarkerbutton = widgets.Button(description="delete last")
output = widgets.Output()

def on_deletemarkerbutton_clicked(b):
    if len(markers)!=0:
        del markers[-1]
    with output:
        print("Markers:",markers)

deletemarkerbutton.on_click(on_deletemarkerbutton_clicked)



showmarkersbutton = widgets.Button(description="show markers", button_style='info')

def on_showmarkersbutton_clicked(b):
    with output:
        print("Markers:",markers)
showmarkersbutton.on_click(on_showmarkersbutton_clicked)

###################################################

widgets.HBox([markernumber,addmarkerbutton,deletemarkerbutton,showmarkersbutton,output])
#########################

Enter the marker labels:


# Patterns and Markers

If you've used the widgets above to speficy your pattern and makers, **skip the cell below.**

If not: you can specify the strings named "pattern" and "markers" on your own below. If you're comfortable editing them this way, I recommend using this, instead of the widgets above. It's faster and you can save your work.

*  For standard components, use entries from these options: S, IQ, OQ, IP, OP, IT, OT (Straight, Inside Quarter turn, Outside Quarter turn, Inside Point, Outside Point, Inside Twist, Outside Twist). Examples are commented-out for you to see.
*  For custom instructions, you can enter a list. The program will print the 0th entry for the foundation chain, the next entry for the 1st row, etc. So you can enter the list as e.g. ["instructions for row 0","instructions for row 1"]
*  For runs, there's a program that will make a list (as in the previous bullet point) for you. The synatx is: runs(list), where the list alternates between corner types and run lengths. For example:
  *  run(["ID",3,"IJ"])
  *  run(["ID",5,"IC",4,"OJ"])
  *  run(["OJ",4,"IL",5,"IC",6,"OJ"])

In [8]:
#########################
##border example knot
#########################
#Twists:
#pattern=["S","S","OP","S","S","IT","S","S","OT","S","S","IP","S","S"]
#markers=["1T","2B","3T","4B","5T","5B","6T","5B","5T","4B","3T","2B","1T","end"]


#Ovals:
#pattern=["S","IP","S","S","S","IP","S","S",]
#markers=["2T","1B","4T","3B","2T","1B","4T","end"]


#End Pieces:
#pattern=["S","IP","S","IQ","OT","S"]
#markers=["3T","2B","1T","7B","7T","end"]

#########################
##rectangle example knot
#########################
#pattern=["S","IQ","S","IP","IQ","IP","S","IQ","S","S","OQ","OQ","OQ","S"]
#markers=["1T","2B","3T","4B","2T","3B","4T","5B","6T","7B","5T","1B","7T"]

# Generate Pattern

Use shift+enter on the cell below to generate your crochet pattern!

This uses all the things you've specified, from the size of your stitches to the pattern of your knot. If you change one of them, go through the notebook again until you make it back here.

In [15]:
##check if the markers are too short. 
#If there's been no attempt to define markers, it'll be an empty string.
#If there are some markers, it will add question marks
if len(markers)==0:
    for i in range(len(pattern)):
        markers.append("")
elif len(markers)<len(pattern)-1:#in this case there are some markers but not enough. Add question marks so it's clear these aren't intentional blanks
    for i in range(len(pattern)-len(markers)):
        markers.append(".")

#pattern printing starts here

for r in range(TotalRows+1):
        #print row number
        #foundation row uses chains, other rows use sc
        if r==0:
            print("***Foundation Chain***")
            stitch="ch"
        else:
            print()
            
            print("***Row "+str(r)+"***")
            stitch="sc"
        if (r%2)==0:#even rows and odd rows work the pattern in opposite directions
            marktrack = 0 #markers[0] starts the row
            markincrement=1
        else:
            marktrack = len(pattern)-2 #markers[n] starts the row
            markincrement=-1
        #even rows: pattern[0], pattern[1]. 
        #odd rows: pattern[n],pattern[n-1]. 
        for SegmentNumber in range(len(pattern)):
            if (r % 2) == 0:
                seg = pattern[SegmentNumber]
                mark=markers[SegmentNumber] #mark that goes at the end of the component
            else:
                seg = pattern[len(pattern)-SegmentNumber-1]
                mark = markers[len(pattern)-SegmentNumber-2]
        ###how to print markers
            if SegmentNumber==len(pattern)-1: #no marker at end of row
                    end=""
            else:
                    end="/("+mark+")"
        #print instructions for each component
        #########################################################################
            ###straight
            if seg == "S":
                print (dtd,stitch,end)
            ###quarter turns
            elif seg=="IQ" or seg=="OQ":
                if seg=="IQ":
                    SegmentSC=QuarterTurn[r]
                    if r!=0:
                        SegmentChange="+"
                        SegmentDelta=QuarterTurn[r]-QuarterTurn[r-1]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                elif seg=="OQ":
                    SegmentSC=QuarterTurn[TotalRows-r]
                    if r!=0:
                        SegmentChange="-"
                        SegmentDelta=QuarterTurn[TotalRows-(r-1)]-QuarterTurn[TotalRows-r]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                if r==0:
                    if QTs>0:
                        print(QTs,stitch,"/",SegmentSC,stitch,"/",QTs,stitch,end)
                    else:
                        print(SegmentSC,stitch,end)

                else:
                    if QTs>0:
                        print(QTs,stitch,"/",SegmentSC,stitch+change,"/",QTs,stitch,end)
                    else:
                        print(SegmentSC,stitch+change,end)
            ###points
            elif seg=="IP" or seg=="OP":
                if seg=="IP":
                    SegmentSC=PointArc[r]
                    CornerStyle="2 ch,"
                    if r!=0:
                        #arc part
                        SegmentChange="+"
                        SegmentDelta=PointArc[r]-PointArc[r-1]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                        #straight part
                        #PointCorner gives the measure of a side; one stitch will be a chain, the others sc
                        StraightPart=PointCorner[r]-1                         

                    else: #foundation chain row: both sides consist of chains
                        StraightPart=2*PointCorner[r]

                elif seg=="OP":
                    SegmentSC=PointArc[TotalRows-r]
                    CornerStyle="2 dec sts,"
                    if r!=0:
                        #arc part
                        SegmentChange="-"
                        SegmentDelta=PointArc[TotalRows-(r-1)]-PointArc[TotalRows-r]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                        #straight part
                        #PointCorner gives the measure of a side; one stitch will be a chain, the others sc
                        StraightPart=PointCorner[TotalRows-r]-1 
                    else:#foundation chain row: everything is a chain
                        StraightPart=2*PointCorner[TotalRows-r]
                if r==0:
                    print(SegmentSC,"ch /",StraightPart,"ch /",SegmentSC,end)
                else:
                    print(SegmentSC,"sc"+change,"/",StraightPart,"sc,",CornerStyle,StraightPart,"sc /",SegmentSC,"sc "+change,end)
            ###Twists
            elif seg=="IT" or seg=="OT":
                if TS==0:
                    Tmiddle=""
                else:
                    Tmiddle=str(TS)+" "+stitch+" /"
                if seg=="IT":
                    SegmentSCOuter=PointArc[r]
                    SegmentSCInner=TwistTop[r]
                    if r!=0:
                        SegmentChange="+"
                        SegmentDeltaOuter=PointArc[r]-PointArc[r-1]
                        SegmentDeltaInner=TwistTop[r]-TwistTop[r-1]
                        if SegmentDeltaOuter==0:
                            changeOuter=""
                        else:
                            changeOuter="("+SegmentChange+str(SegmentDeltaOuter)+")"
                        if SegmentDeltaInner==0:
                            changeInner=""
                        else:
                            changeInner="("+SegmentChange+str(SegmentDeltaInner)+")"
                elif seg=="OT":
                    SegmentSCOuter=PointArc[TotalRows-r]
                    SegmentSCInner=TwistTop[TotalRows-r]
                    if r!=0:
                        SegmentChange="-"
                        SegmentDeltaOuter=PointArc[TotalRows-(r-1)]-PointArc[TotalRows-r]
                        SegmentDeltaInner=TwistTop[TotalRows-(r-1)]-TwistTop[TotalRows-r]
                        if SegmentDeltaOuter==0:
                            changeOuter=""
                        else:
                            changeOuter="("+SegmentChange+str(SegmentDeltaOuter)+")"
                        if SegmentDeltaInner==0:
                            changeInner=""
                        else:
                            changeInner="("+SegmentChange+str(SegmentDeltaInner)+")"

                if r==0:
                    print(SegmentSCOuter,stitch,"/",Tmiddle,SegmentSCInner,stitch,"/",Tmiddle,SegmentSCOuter,end)
                else:
                    print(SegmentSCOuter,stitch,changeOuter,"/",Tmiddle,SegmentSCInner,stitch,changeInner,"/",Tmiddle,SegmentSCOuter,"sc "+changeOuter,end)
            ###Explicitly-written strings (incl runs)
            else:
                print(seg[r],end)
print()            
print("Finish off")

***Foundation Chain***
10 ch /(1T)
3 ch / 7 ch / 3 ch /(.)
3 ch / 15 ch / 3 ch /(.)
6 ch / 2 ch / 6 /(.)
10 ch / 14 ch / 10 /(.)
6 ch /  2 ch /  6 /(.)


IndexError: list index out of range

# Yardage

Work a swatch that is the equvalent of one straight section, and measure the length of yarn you used. The code cell below will tell you how many times that amount you'll need. For example, if you find that a straight section takes 10 metres, and the code below tells you your yardage is 15 times a straight section, that means your total yardage is about 150 metres.

*As always, yardage is approximate!* This is my best guess, but it's good to have extra on hand.

If you don't have runs in your pattern, this should work fine with minimal effort.

If you do have runs, it's slightly more complicated. The stitch counts are computed using global variables that don't reset until you reset the entire notebook. So if you run this cell twice, the numbers will change. The best way to use this is to write all of your values into the code (not using widgets), then hit the double-arrow button to run everything from scratch precisely once. Or, if you only have a small number of short runs, assum that they will add only a very small percentage to the total.

In [10]:
NoRunCount=0
NoRunOCount=0
#add up all the 
for r in pattern:
    if r=="S":
        TotalCount=TotalCount+SCount
        TotalOCount=TotalOCount+SOCount
        NoRunCount=NoRunCount+SCount
        NoRunOCount=NoRunOCount+SOCount
    elif r[1]=="Q":
        TotalCount=TotalCount+QCount
        TotalOCount=TotalOCount+QOCount
        NoRunCount=NoRunCount+QCount
        NoRunOCount=NoRunOCount+QOCount
    elif r[1]=="P":
        TotalCount=TotalCount+PCount
        TotalOCount=TotalOCount+POCount
        NoRunCount=NoRunCount+PCount
        NoRunOCount=NoRunOCount+POCount
    elif r[1]=="T":
        TotalCount=TotalCount+TCount
        TotalOCount=TotalOCount+TOCount
        NoRunCount=NoRunCount+TCount
        NoRunOCount=NoRunOCount+TOCount
MainYardage=round(TotalCount/SCount)
OYardage=round(TotalOCount/SOCount)
NoRunYardage=round(NoRunCount/SCount)
NoRunOYardage=round(NoRunOCount/SOCount)
print("**Without Runs**")
print("If your piece has no runs, then you should need about",NoRunYardage,"times the amount of yarn used in a straight section ("+str(TotalRows),"rows of",dtd,"sc) for the main bodies of the strands, and about",NoRunOYardage,"times the amount of yarn used in the outline of a straight section ("+str(2*dtd),"slip stitches)")
print()
print("**With Runs**")
print("**If you want to use the estimates below, run this notebook exactly once from start to finish.**")
print("Yardage is about",MainYardage,"times that needed for a straight section ("+str(TotalRows),"rows of",dtd,"sc)")
print("Outline yardage is ",OYardage,"times that needed for a straight section (2 rows of",dtd,"slip stitches)")


**Without Runs**
If your piece has no runs, then you should need about 0 times the amount of yarn used in a straight section (6 rows of 10 sc) for the main bodies of the strands, and about 0 times the amount of yarn used in the outline of a straight section (20 slip stitches)

**With Runs**
**If you want to use the estimates below, run this notebook exactly once from start to finish.**
Yardage is about 0 times that needed for a straight section (6 rows of 10 sc)
Outline yardage is  0 times that needed for a straight section (2 rows of 10 slip stitches)


# 